In [1]:
surname = "Петров" #Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  3
Задача № 2:  1


In [2]:
from pyspark.sql import SparkSession, functions as F, Window


spark = SparkSession.builder.appName("MovieLens").getOrCreate()
base_path = "ml-latest-small"
movies = spark.read.csv(f"{base_path}/movies.csv", header=True, inferSchema=True)
ratings = spark.read.csv(f"{base_path}/ratings.csv", header=True, inferSchema=True)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/18 16:30:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Анализ датасета

In [3]:
ratings.show(3)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
+------+-------+------+---------+
only showing top 3 rows


In [4]:
movies_exploded = (
    movies.select("movieId", "title", F.explode(F.split(F.col("genres"), "\|")).alias("genre"), 
                  F.size(F.split("genres", "\|")).alias("genre_count")
).filter(F.col("genre") != "(no genres listed)"))

movies_exploded.show(5)

+-------+----------------+---------+-----------+
|movieId|           title|    genre|genre_count|
+-------+----------------+---------+-----------+
|      1|Toy Story (1995)|Adventure|          5|
|      1|Toy Story (1995)|Animation|          5|
|      1|Toy Story (1995)| Children|          5|
|      1|Toy Story (1995)|   Comedy|          5|
|      1|Toy Story (1995)|  Fantasy|          5|
+-------+----------------+---------+-----------+
only showing top 5 rows


In [5]:
genre_movie_counts = (
    movies_exploded
        .groupBy("genre")
        .agg(F.countDistinct("movieId").alias("movie_count"))
        .orderBy("genre")
)
genre_movie_counts.show()

+-----------+-----------+
|      genre|movie_count|
+-----------+-----------+
|     Action|       1828|
|  Adventure|       1263|
|  Animation|        611|
|   Children|        664|
|     Comedy|       3756|
|      Crime|       1199|
|Documentary|        440|
|      Drama|       4361|
|    Fantasy|        779|
|  Film-Noir|         87|
|     Horror|        978|
|       IMAX|        158|
|    Musical|        334|
|    Mystery|        573|
|    Romance|       1596|
|     Sci-Fi|        980|
|   Thriller|       1894|
|        War|        382|
|    Western|        167|
+-----------+-----------+



In [6]:
my_genres = ['Thriller', 'Sci-Fi', 'Adventure']

movie_stats_0 = (
    ratings
    .groupBy("movieId")
    .agg(
        F.count(F.lit(1)).alias("rating_cnt"),
        F.avg("rating").alias("avg_rating")
    )
    .join(
        movies.select("movieId", "title", F.split(F.col("genres"), "\|").alias("genres_arr")),
        "movieId", "inner"
    )
    .where(F.array_contains(F.col("genres_arr"), my_genres[0]))
    .orderBy(F.desc("rating_cnt"), F.desc("avg_rating"), F.asc("title"))
)

print(f'{my_genres[0]} топ 10 фильмов с наибольшим колвом рейтингов')
movie_stats_0.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")).show(10)

print(f'\n{my_genres[0]} топ 10 фильмов с наименьшим колвом рейтингов')
movie_stats_0.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.asc("rating_cnt"), F.desc("avg_rating"), F.asc("title")).show(10)

print(f'\n{my_genres[0]} топ 10 фильмов с наибольшим средним рейтингом')
movie_stats_0.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.desc("avg_rating"), F.asc("title")).show(10)

print(f'\n{my_genres[0]} топ 10 фильмов с наименьшим средним рейтингом')
movie_stats_0.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.asc("avg_rating"), F.asc("title")).show(10)

Thriller топ 10 фильмов с наибольшим колвом рейтингов
+-------+--------------------+----------+--------------------+----------+
|movieId|               title|rating_cnt|          genres_arr|avg_rating|
+-------+--------------------+----------+--------------------+----------+
|    296| Pulp Fiction (1994)|       307|[Comedy, Crime, D...|     4.197|
|    593|Silence of the La...|       279|[Crime, Horror, T...|     4.161|
|   2571|  Matrix, The (1999)|       278|[Action, Sci-Fi, ...|     4.192|
|    480|Jurassic Park (1993)|       238|[Action, Adventur...|      3.75|
|   2959|   Fight Club (1999)|       218|[Action, Crime, D...|     4.273|
|     50|Usual Suspects, T...|       204|[Crime, Mystery, ...|     4.238|
|     47|Seven (a.k.a. Se7...|       203| [Mystery, Thriller]|     3.975|
|    780|Independence Day ...|       202|[Action, Adventur...|     3.446|
|    457|Fugitive, The (1993)|       190|          [Thriller]|     3.992|
|    592|       Batman (1989)|       189|[Action, Crime, T

In [7]:
movie_stats_1 = (
    ratings
    .groupBy("movieId")
    .agg(
        F.count(F.lit(1)).alias("rating_cnt"),
        F.avg("rating").alias("avg_rating")
    )
    .join(
        movies.select("movieId", "title", F.split(F.col("genres"), "\|").alias("genres_arr")),
        "movieId", "inner"
    )
    .where(F.array_contains(F.col("genres_arr"), my_genres[1]))
    .orderBy(F.desc("rating_cnt"), F.desc("avg_rating"), F.asc("title"))
)

print(f'{my_genres[1]} топ 10 фильмов с наибольшим колвом рейтингов')
movie_stats_1.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")).show(10)

print(f'\n{my_genres[1]} топ 10 фильмов с наименьшим колвом рейтингов')
movie_stats_1.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.asc("rating_cnt"), F.desc("avg_rating"), F.asc("title")).show(10)

print(f'\n{my_genres[1]} топ 10 фильмов с наибольшим средним рейтингом')
movie_stats_1.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.desc("avg_rating"), F.asc("title")).show(10)

print(f'\n{my_genres[1]} топ 10 фильмов с наименьшим средним рейтингом')
movie_stats_1.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.asc("avg_rating"), F.asc("title")).show(10)

Sci-Fi топ 10 фильмов с наибольшим колвом рейтингов
+-------+--------------------+----------+--------------------+----------+
|movieId|               title|rating_cnt|          genres_arr|avg_rating|
+-------+--------------------+----------+--------------------+----------+
|   2571|  Matrix, The (1999)|       278|[Action, Sci-Fi, ...|     4.192|
|    260|Star Wars: Episod...|       251|[Action, Adventur...|     4.231|
|    480|Jurassic Park (1993)|       238|[Action, Adventur...|      3.75|
|    589|Terminator 2: Jud...|       224|    [Action, Sci-Fi]|     3.971|
|   1196|Star Wars: Episod...|       211|[Action, Adventur...|     4.216|
|    780|Independence Day ...|       202|[Action, Adventur...|     3.446|
|   1210|Star Wars: Episod...|       196|[Action, Adventur...|     4.138|
|     32|Twelve Monkeys (a...|       177|[Mystery, Sci-Fi,...|     3.983|
|   1270|Back to the Futur...|       171|[Adventure, Comed...|     4.038|
|   1580|Men in Black (a.k...|       165|[Action, Comedy, ..

In [8]:
movie_stats_2 = (
    ratings
    .groupBy("movieId")
    .agg(
        F.count(F.lit(1)).alias("rating_cnt"),
        F.avg("rating").alias("avg_rating")
    )
    .join(
        movies.select("movieId", "title", F.split(F.col("genres"), "\|").alias("genres_arr")),
        "movieId", "inner"
    )
    .where(F.array_contains(F.col("genres_arr"), my_genres[2]))
    .orderBy(F.desc("rating_cnt"), F.desc("avg_rating"), F.asc("title"))
)

print(f'{my_genres[2]} топ 10 фильмов с наибольшим колвом рейтингов')
movie_stats_2.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")).show(10)

print(f'\n{my_genres[2]} топ 10 фильмов с наименьшим колвом рейтингов')
movie_stats_2.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.asc("rating_cnt"), F.desc("avg_rating"), F.asc("title")).show(10)

print(f'\n{my_genres[2]} топ 10 фильмов с наибольшим средним рейтингом')
movie_stats_2.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.desc("avg_rating"), F.asc("title")).show(10)

print(f'\n{my_genres[2]} топ 10 фильмов с наименьшим средним рейтингом')
movie_stats_2.select("movieId", "title", "rating_cnt", 'genres_arr',F.round("avg_rating", 3).alias("avg_rating")) \
    .where(F.col("rating_cnt") > 10) \
    .orderBy(F.asc("avg_rating"), F.asc("title")).show(10)

Adventure топ 10 фильмов с наибольшим колвом рейтингов
+-------+--------------------+----------+--------------------+----------+
|movieId|               title|rating_cnt|          genres_arr|avg_rating|
+-------+--------------------+----------+--------------------+----------+
|    260|Star Wars: Episod...|       251|[Action, Adventur...|     4.231|
|    480|Jurassic Park (1993)|       238|[Action, Adventur...|      3.75|
|      1|    Toy Story (1995)|       215|[Adventure, Anima...|     3.921|
|   1196|Star Wars: Episod...|       211|[Action, Adventur...|     4.216|
|    780|Independence Day ...|       202|[Action, Adventur...|     3.446|
|    150|    Apollo 13 (1995)|       201|[Adventure, Drama...|     3.846|
|   1198|Raiders of the Lo...|       200| [Action, Adventure]|     4.208|
|   4993|Lord of the Rings...|       198|[Adventure, Fantasy]|     4.106|
|   1210|Star Wars: Episod...|       196|[Action, Adventur...|     4.138|
|   5952|Lord of the Rings...|       188|[Adventure, Fant

# Colaborative filtering
## User-based

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator

train_init, test = ratings.randomSplit([0.8, 0.2], seed=42)
train_init.cache()
test.cache()

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [10]:
global_mean = train_init.agg(F.avg("rating").alias("mean")).first()["mean"]

test_baseline = test.withColumn("pred", F.lit(global_mean))
rmse_eval = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="pred")
rmse_baseline = rmse_eval.evaluate(test_baseline)
print(f"Baseline RMSE: {rmse_baseline:.4f}")

user_mean = train_init.groupBy("userId").agg(F.avg("rating").alias("user_mean"))
item_mean = train_init.groupBy("movieId").agg(F.avg("rating").alias("item_mean"))

Baseline RMSE: 1.0504


In [11]:
r = train_init.select("userId", "movieId", "rating")
r1 = r.alias("r1")
r2 = r.alias("r2")

user_pairs = (
    r1.join(r2, on="movieId")
    .where(F.col("r1.userId") < F.col("r2.userId"))
    .select(
        F.col("r1.movieId").alias("movieId"),
        F.col("r1.userId").alias("u1"),
        F.col("r2.userId").alias("u2"),
        F.col("r1.rating").alias("u1_rating"),
        F.col("r2.rating").alias("u2_rating")
    )
)

user_pairs.show(10)


+-------+---+---+---------+---------+
|movieId| u1| u2|u1_rating|u2_rating|
+-------+---+---+---------+---------+
|      1|  1|609|      4.0|      3.0|
|      1|  1|608|      4.0|      2.5|
|      1|  1|607|      4.0|      4.0|
|      1|  1|606|      4.0|      2.5|
|      1|  1|604|      4.0|      3.0|
|      1|  1|603|      4.0|      4.0|
|      1|  1|601|      4.0|      4.0|
|      1|  1|600|      4.0|      2.5|
|      1|  1|597|      4.0|      4.0|
|      1|  1|596|      4.0|      4.0|
+-------+---+---+---------+---------+
only showing top 10 rows


In [12]:
topK = 50
min_common = 2
shrink = 10.0

user_sims = (
    user_pairs.groupBy(F.col("u1"), F.col("u2"))
    .agg(
        F.sum(F.col("u1_rating") * F.col("u2_rating")).alias("dot"),
        F.sum(F.col("u1_rating") * F.col("u1_rating")).alias("n1"),
        F.sum(F.col("u2_rating") * F.col("u2_rating")).alias("n2"),
        F.count("*").alias("cnt")
    )
    .filter(F.col("cnt") >= min_common)
    .withColumn("sim_raw", F.col("dot") / (F.sqrt(F.col("n1")) * F.sqrt(F.col("n2"))))
    .withColumn("sim", F.col("sim_raw") * (F.col("cnt") / (F.col("cnt") + F.lit(shrink))))
    .select("u1", "u2", "sim")
)

user_sims_long = (
    user_sims.select(F.col("u1").alias("u1"), F.col("u2").alias("u2"), "sim")
    .unionByName(user_sims.select(F.col("u2").alias("u1"), F.col("u1").alias("u2"), "sim"))
).cache()

user_sims_long.show(10)

+---+---+-------------------+
| u1| u2|                sim|
+---+---+-------------------+
|  1|587| 0.6098803142005349|
|  2|582| 0.4046083186874731|
|  4|599| 0.8180343420964864|
|  7|475| 0.7002520719362795|
|  9|564|0.16499158227686106|
| 16|584| 0.2239819004524887|
| 18|552|  0.726933525611359|
| 21|600| 0.7084129850543538|
| 26|594| 0.2729128267406192|
| 28|571| 0.3450714592698917|
+---+---+-------------------+
only showing top 10 rows


In [13]:
from pyspark.sql.functions import broadcast


w = Window.partitionBy("u1").orderBy(F.desc("sim"))
user_sims_topk = (
    user_sims_long
    .filter(F.col("sim") > 0)
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") <= topK)
    .select("u1", "u2", "sim")
    .cache()
)

user_sims_topk.show(20)

+---+---+------------------+
| u1| u2|               sim|
+---+---+------------------+
|148|249| 0.696537459041315|
|148|610|0.6739249580442203|
|148|596|0.6641007754922876|
|148|414|0.6522506018821762|
|148| 21|0.6470136907838365|
|148|525|0.6423553330433289|
|148| 68|0.6405867395610184|
|148|105|0.6340403525576503|
|148|212|0.6339675390891377|
|148|599|0.6289741930413346|
|148|380|0.6222113741972644|
|148|247|0.6186704046854858|
|148| 62|0.6077011661815186|
|148|483|0.6067008496999003|
|148|177|0.6057357517715113|
|148|534|0.6042360596728864|
|148|509|0.6041402701903863|
|148| 18|0.5919904911450831|
|148|318|0.5909389871116821|
|148|567|0.5868512724059924|
+---+---+------------------+
only showing top 20 rows


In [14]:
t = test.select("userId", "movieId", "rating").alias("t")
tr = train_init.select("userId", "movieId", "rating").alias("tr")
s = user_sims_topk.alias("s")

predictions_base = (
    t.join(s, t.userId == s.u1, "left")
    .join(tr, (F.col("s.u2") == F.col("tr.userId")) & (t.movieId == F.col("tr.movieId")), "left")
    .select(
        F.col("t.userId").alias("userId"),
        F.col("t.movieId").alias("movieId"),
        F.col("t.rating").alias("actual_rating"),
        F.col("s.sim").alias("sim"),
        F.col("tr.rating").alias("neighbor_rating")
    )
)

predictions = (
    predictions_base
    .filter(F.col("neighbor_rating").isNotNull())
    .groupBy("userId", "movieId", "actual_rating")
    .agg(
        F.sum(F.col("sim") * F.col("neighbor_rating")).alias("w_sum"),
        F.sum(F.col("sim")).alias("sim_sum")
    )
    .withColumn("pred", 
        F.when(F.col("sim_sum") > 0, F.col("w_sum") / F.col("sim_sum"))
        .otherwise(F.lit(global_mean))
    )
    .select("userId", "movieId", "actual_rating", "pred")
)

In [15]:
rmse_pred = rmse_eval.evaluate(predictions.withColumnRenamed("actual_rating", "rating"))

print(f"Baseline RMSE: {rmse_baseline:.4f}")
print(f"User-based CF RMSE: {rmse_pred:.4f}")
print(f"Улучшение: {((rmse_baseline - rmse_pred) / rmse_baseline * 100):.2f}%")

Baseline RMSE: 1.0504
User-based CF RMSE: 0.9692
Улучшение: 7.73%


# ALS

In [16]:
tr.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    110|   4.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    356|   4.0|
|     1|    362|   5.0|
|     1|    367|   4.0|
|     1|    441|   4.0|
|     1|    457|   5.0|
+------+-------+------+
only showing top 20 rows


In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import numpy as np

rank_values = [5, 10, 15]
reg_values = [0.001, 0.01, 0.1, 1, 10]

als = ALS(
    userCol="userId",
    itemCol="movieId", 
    ratingCol="rating",
    coldStartStrategy="drop",
    seed=42
)

rmse_eval_als = RegressionEvaluator(
    metricName="rmse", 
    labelCol="rating", 
    predictionCol="prediction" 
)

paramGrid = (
    ParamGridBuilder()
    .addGrid(als.rank, rank_values)
    .addGrid(als.regParam, reg_values)
    .build()
)

crossval = CrossValidator(
    estimator=als,
    estimatorParamMaps=paramGrid,
    evaluator=rmse_eval_als,
    numFolds=4,
    seed=42,
    parallelism=2
)

cv_model = crossval.fit(tr)


25/11/18 16:32:27 WARN CacheManager: Asked to cache already cached data.
25/11/18 16:32:27 WARN CacheManager: Asked to cache already cached data.


In [ ]:
best_model = cv_model.bestModel
print(f"\nЛучшие параметры:")
print(f"факторы: {best_model.rank}")
print(f"регуляризация: {best_model._java_obj.parent().getRegParam()}")

als_predictions = best_model.transform(test)
rmse_als = rmse_eval_als.evaluate(als_predictions)

print(f"Baseline RMSE:\t\t\t{rmse_baseline:.4f}")
print(f"User-based CF RMSE:\t\t{rmse_pred:.4f} (улучшение: {((rmse_baseline - rmse_pred) / rmse_baseline * 100):.2f}%)")
print(f"ALS RMSE:\t\t\t{rmse_als:.4f} (улучшение: {((rmse_baseline - rmse_als) / rmse_baseline * 100):.2f}%)")



Лучшие параметры:
факторы: 5
регуляризация: 0.1
Baseline RMSE:			1.0504
User-based CF RMSE:		0.9692 (улучшение: 7.73%)
ALS RMSE:			0.8756 (улучшение: 16.65%)


25/11/18 17:35:07 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 274139 ms exceeds timeout 120000 ms
25/11/18 17:35:07 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/18 17:35:16 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$